In [1]:
import torch

# 手書き数字の画像データMNISTをダウンロード
 
 
from sklearn.datasets import fetch_openml
import numpy as np

mnist_X, mnist_y = fetch_openml('mnist_784', version=1, data_home=".", return_X_y=True)

x = mnist_X.astype(np.float32) / 255
y = mnist_y.astype(np.int32)

c:\Users\ikemix\anaconda3\envs\rl_env\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


ここからPyTorchによるディープラーニングを実装します。今回は最低限の実装であるミニマムバージョンで解説します。PyTorchによるディープラーニングの実装は次の6ステップで行われます。

データの前処理

DataLoderの作成

ネットワークの構築

誤差関数と最適化手法の設定

学習と推論の設定

学習と推論の実行

データの前処理では、データをニューラルネットワークに投入できるように加工します。今回のMNISTデータの場合は、まずダウンロードしたデータを画像Xとラベルy（0-9）に分けて格納します。画像データはグレースケールの0-255の数値で表現されているので、255で割って0-1になるように正規化します。


In [3]:


# MNISTのデータの1つ目を可視化する
 
import matplotlib.pyplot as plt
% matplotlib inline
 
plt.imshow(X[0].reshape(28, 28), cmap='gray')
print("この画像データのラベルは{:.0f}です".format(y[0]))

"""すると以下の図12.3の結果が出力されます。
変数Xには縦28×横28ピクセルの784の要素を持つNumpy形式のベクトルが7万画像分格納されています。"""

UsageError: Line magic function `%` not found.


続いて正規化したMNISTデータをPyTorchのニューラルネットワークで扱えるDataLoaderという変数へと変換します。

　DataLoaderへの変換は以下の4つの手続きからなります。

2.1. 訓練データとテストデータに分離
2.2. NumPyデータをTensorに変換
2.3. Datasetの作成
2.4. DatasetをDataLoderに変換

In [4]:
# 2. DataLoderの作成
 
import torch
from torch.utils.data import TensorDataset, DataLoader
 
# 2.1 データを訓練とテストに分割（6:1）
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1/7, random_state=0)
 
# 2.2 データをPyTorchのTensorに変換
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
 
# 2.3 データとラベルをセットにしたDatasetを作成
ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)
 
# 2.4 データセットのミニバッチサイズを指定した、Dataloaderを作成
# Chainerのiterators.SerialIteratorと似ている
loader_train = DataLoader(ds_train, batch_size=64, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=, shuffle=False)

SyntaxError: invalid syntax (3539852970.py, line 24)

In [ ]:

# 3. ネットワークの構築
# Keras風の書き方 Define and Run
 
from torch import nn
 
model = nn.Sequential()
model.add_module('fc1', nn.Linear(28*28, 100))
model.add_module('relu1', nn.ReLU())
model.add_module('fc2', nn.Linear(100, 100))
model.add_module('relu2', nn.ReLU())
model.add_module('fc3', nn.Linear(100, 10))
 
print(model)

In [2]:

# 4. 誤差関数と最適化手法の設定
 
from torch import optim
 
# 誤差関数の設定
loss_fn = nn.CrossEntropyLoss()  # 変数名にはcriterionも使われる
 
# 重みを学習する際の最適化手法の選択
optimizer = optim.Adam(model.parameters(), lr=0.01)

NameError: name 'nn' is not defined

In [ ]:
# 5. 学習と推論の設定
# 5-1. 学習1回でやることを定義します
# Chainerのtraining.Trainer()に対応するものはない
 
from torch.autograd import Variable
 
 
def train(epoch):
    model.train()  # ネットワークを学習モードに切り替える
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in loader_train:
        data, target = Variable(data), Variable(target)  # 微分可能に変換
        optimizer.zero_grad()  # 一度計算された勾配結果を0にリセット
 
        output = model(data)  # 入力dataをinputし、出力を求める
        loss = loss_fn(output, target)  # 出力と訓練データの正解との誤差を求める
        loss.backward()  # 誤差のバックプロパゲーションを求める
        optimizer.step()  # バックプロパゲーションの値で重みを更新する
 
    print("epoch{}：終了\n".format(epoch))

In [ ]:
# 5. 学習と推論の設定
# 5-2. 推論1回でやることを定義します
# Chainerのtrainer.extend(extensions.Evaluator())に対応するものはない
 
 
def test():
    model.eval()  # ネットワークを推論モードに切り替える
    correct = 0
 
    # データローダーから1ミニバッチずつ取り出して計算する
    for data, target in loader_test:
        data, target = Variable(data), Variable(target)  # 微分可能に変換
        output = model(data)  # 入力dataをinputし、出力を求める
 
        # 推論する
        pred = output.data.max(1, keepdim=True)[1]  # 出力ラベルを求める
        correct += pred.eq(target.data.view_as(pred)).sum()  # 正解と一緒だったらカウントアップ
 
    # 正解率を出力
    data_num = len(loader_test.dataset)  # データの総数
    print('\nテストデータの正解率: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))


In [ ]:
# 学習なしにテストデータで推論してみよう
test()